## Домашнее задание:
С помощью бертов решить задачу классификации.

In [ ]:
!pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b7f2129279a5fbfbbbc446adde5a016735ce97d22d31de1574c10acbc1a1d111
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import datasets
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer

from transformers import DataCollatorWithPadding

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'paraphrases'],
        num_rows: 1725393
    })
})

In [ ]:
corpus['train']

Dataset({
    features: ['id', 'paraphrases'],
    num_rows: 1725393
})

In [ ]:
corpus['train'][:1]

{'id': ['0'],
 'paraphrases': [['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
   'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем']]}

In [ ]:
corpus['train']['paraphrases'][0]

['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
 'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем']

In [ ]:
corpus['train']['paraphrases'][0][0]

'Птичий праздник: Что такое "куриное рождество" и кто его отмечает'

# Используем предобученную модель: cointegrated/rubert-base-cased-dp-paraphrase-detection
### https://huggingface.co/cointegrated/rubert-base-cased-dp-paraphrase-detection

In [ ]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()

    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"

    return [text1, text2, p[0], p[1], paraphrase]


print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))

print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))


['Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода', 0.70562273, 0.29437727, 'non']
['Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась', 0.16524385, 0.83475614, 'paraphrase']


In [ ]:
def compare(text1, text2):
    """  """

    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()

    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    paraphrase = "non"

    if p[1] > 0.5:
        paraphrase = "paraphrase"

    return [text1, text2, p[0], p[1], paraphrase]

In [ ]:
result = []

result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
result.append(compare('Цены на нефть восстанавливаются', 'Парламент Словакии поблагодарил народы'))
result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице прекрасная погода'))
n=5
for i in range(n):
  text1 = corpus['train']['paraphrases'][i][0]
  text2 =corpus['train']['paraphrases'][i][1]
  result.append(compare(text1, text2))



col_names =  ['текст1', 'текст2', 'вероятность non-paraphrase', 'вероятность paraphrase', 'paraphrase']
df = pd.DataFrame(result, columns=col_names)
df.head(10)

,текст1,текст2,вероятность non-paraphrase,вероятность paraphrase,paraphrase
0,Сегодня на улице хорошая погода,Сегодня на улице отвратительная погода,0.705623,0.294377,non
1,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы,0.944955,0.055045,non
2,Сегодня на улице хорошая погода,Сегодня на улице прекрасная погода,0.035242,0.964758,paraphrase
3,"Птичий праздник: Что такое ""куриное рождество"" и кто его отмечает",Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем,0.157879,0.842121,paraphrase
4,В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото),"Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства",0.018209,0.981791,paraphrase
5,"Томас Бах: ""Олимпийский комитет России понес достаточное наказание""",Президент МОК Бах: Россия уже достаточно наказана за допинг,0.028516,0.971484,paraphrase
6,Россия не представила WADA данные по пробам в срок и может быть наказана,WADA: Россия не предоставила нам доступ к допинг-пробам,0.058580,0.941420,paraphrase
7,Антидопинговое агентство США призвало ВАДА отстранить российских спортсменов от соревнований,Комиссия спортсменов WADA потребовала отстранить РУСАДА,0.428032,0.571968,paraphrase


In [ ]:
data = datasets.load_dataset('merionum/ru_paraphraser')

In [ ]:
model_name = "IlyaGusev/xlm_roberta_large_headline_cause_simple"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
label_list = sorted(set(data['train']['class']))
labels2id = { key:id for id, key in enumerate(label_list)}

def tokenize_and_align_labels(tokenizer, labels2id):
    def tokenize_and_align_labels_(examples):
#         tokenized_inputs = tokenizer([[text_1, text_2] for (text_1, text_2) in zip(examples['text_1'],examples['text_2'])])
        tokenized_inputs = tokenizer(examples['text_1'],examples['text_2'], truncation=True)
#         tokenized_inputs["labels"] = [[labels2id[label] for _ in range(len(tokenized_inputs["input_ids"][i]))] for i, label in enumerate(examples['class'])]
        tokenized_inputs["labels"] = [labels2id[label] for label in examples['class']]
#         tokenized_inputs["labels"] = [label for label in examples['class']]
        return tokenized_inputs
    return tokenize_and_align_labels_
tokenized_datasets = data.map(tokenize_and_align_labels(tokenizer, labels2id), batched=True)

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.33.1', '0.22.0')

In [ ]:
! pip install transformers[torch]

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    "paraphras",
    evaluation_strategy = "epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.05,
    save_strategy='no',
    report_to='none',
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

ImportError: ignored

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: id_1, id_2, text_1, class, text_2, id. If id_1, id_2, text_1, class, text_2, id are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7227
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1356
  Number of trainable parameters = 559893507
You're using a XLMRobertaTokenizerFast tokenizer. Please note 

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.735229,0.650208
2,0.758500,0.732371,0.677755
3,0.593200,0.704686,0.700624


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: id_1, id_2, text_1, class, text_2, id. If id_1, id_2, text_1, class, text_2, id are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1924
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: id_1, id_2, text_1, class, text_2, id. If id_1, id_2, text_1, class, text_2, id are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1924
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: id_1, id_2, text_1, class, text_2,

TrainOutput(global_step=1356, training_loss=0.6446539392161862, metrics={'train_runtime': 962.5409, 'train_samples_per_second': 22.525, 'train_steps_per_second': 1.409, 'total_flos': 1819183492473300.0, 'train_loss': 0.6446539392161862, 'epoch': 3.0})

In [ ]:
def get_similarity(text1, text2):
    """ Predict the probability that two Russian sentences are paraphrases of each other. """
    with torch.inference_mode():
        batch = tokenizer(
            text1, text2,
            truncation=True, max_length=model.config.max_position_embeddings, return_tensors='pt',
        ).to(model.device)
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0][1].item()

In [ ]:
text1 = "Я не помню тебя"
text2 = "Я не знаю тебя"
get_similarity(text1, text2)

0.05976682901382446

In [ ]:
text1 = "Это красивая машина"
text2 = "Эта машина выглядит красиво"
get_similarity(text1, text2)

0.7301298379898071

In [ ]:
text1 = "Твой отец говорил, что не любит футбол"
text2 = "Твой отец рассказал о своей нелюбви к футболу"
get_similarity(text1, text2)

0.6921333074569702

In [ ]:
text1 = "Моя сестра любит животных"
text2 = "Моя сестра любит собаку"
get_similarity(text1, text2)

0.49897482991218567

In [ ]:
text1 = "Моя сестра любит животных"
text2 = "Моя сестра любит программировать на Python"
get_similarity(text1, text2)

0.1102612093091011

In [ ]:
text1 = "Моя сестра любит животных"
text2 = "Моя сестра любит программировать"
get_similarity(text1, text2)

0.08786703646183014

In [ ]:
id2labels = { id:key for id, key in enumerate(label_list)}
example = tokenized_datasets["test"][0:15]
tokens = tokenizer(example['text_1'], example['text_1'], padding=True, truncation=True, return_tensors='pt')
tokens = tokens.to('cuda:0')
with torch.no_grad():
    outputs = model(**tokens)

predicted = outputs.logits.argmax(dim=-1).cpu().numpy()
classes = [id2labels[id_label] for id_label in predicted]
df_example =pd.DataFrame({'text_1':example['text_1'], 'text_2':example['text_2'], 'class':example['class'], 'predict':classes})
df_example

,text_1,text_2,class,predict
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,-1,1
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,-1,1
2,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,-1,1
3,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,-1,1
4,Посол РФ в США: Россия будет бороться с попытк...,Правительство запланировало заработать на лоте...,-1,1
5,Вертолет с 11 иностранцами на борту упал в Пак...,В Пакистане упал вертолет с 11 иностранцами,1,1
6,Самолет вернулся в аэропорт Новосибирска из-за...,Самолет вернулся в новосибирский аэропорт из-з...,1,1
7,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0,1
8,Путин перед Днем Победы подписал указ о присво...,СК РФ: Официальный сайт Следственного комитета...,-1,1
9,Суд оправдал Васильеву в хищении акций на два ...,Суд оправдал Васильеву в хищении акций на 2 мл...,1,1
